In [22]:
import pandas as pd
pd.set_option('display.max_colwidth',1000)
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('violent_crime.csv')

In [3]:
df

,CASE_NUMBER,DATE,BLOCK,PRIMARY_TYPE,UNIQUE_ID,ZIP_CODE,WARD,COMMUNITY_AREA,STREET OUTREACH ORGANIZATION,AREA,...,FBI_CD,FBI_DESCR,IUCR_CD,IUCR_SECONDARY,MONTH,DAY_OF_WEEK,HOUR,LATITUDE,LONGITUDE,LOCATION
0,HZ100096,01/01/2016 02:04:00 AM,4600 S ST LAWRENCE,FATAL SHOOTING,HOM-HZ100096-1,60653,4,GRAND BOULEVARD,None,1,...,01A,HOMICIDE - 1ST OR 2ND DEGREE (INDEX),0110,FIRST DEGREE MURDER,1,6,2,41.810127,-87.611604,POINT (-87.611604058699 41.810126904142)
1,HZ100342,01/01/2016 06:39:00 AM,1900 W GARFIELD BLVD,FATAL SHOOTING,HOM-HZ100342-1,60609,16,NEW CITY,PBMR New City,1,...,01A,HOMICIDE - 1ST OR 2ND DEGREE (INDEX),0110,FIRST DEGREE MURDER,1,6,6,41.794218,-87.673175,POINT (-87.67317465388 41.794218345858)
2,HZ101042,01/01/2016 10:06:00 PM,6100 S SANGAMON ST,FATAL SHOOTING,HOM-HZ101042-1,60621,16,ENGLEWOOD,TADC Englewood | READI 60621,1,...,01A,HOMICIDE - 1ST OR 2ND DEGREE (INDEX),0110,FIRST DEGREE MURDER,1,6,22,41.782338,-87.647905,POINT (-87.64790465388 41.782338345858)
3,HZ101842,01/02/2016 04:40:00 PM,6300 S STONY ISLAND,FATAL SHOOTING,HOM-HZ101842-1,60637,20,WOODLAWN,Project H.O.O.D. Woodlawn,1,...,01A,HOMICIDE - 1ST OR 2ND DEGREE (INDEX),0110,FIRST DEGREE MURDER,1,7,16,41.780618,-87.586165,POINT (-87.58616465388 41.780618345858)
4,HZ103309,01/04/2016 08:22:00 AM,400 N CENTRAL AVE,FATAL SHOOTING,HOM-HZ103309-1,60644,37,AUSTIN,READI 60644 | INVC Austin,4,...,01A,HOMICIDE - 1ST OR 2ND DEGREE (INDEX),0110,FIRST DEGREE MURDER,1,2,8,41.888383,-87.765249,POINT (-87.765249058699 41.888383095858)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20031,JE266314,06/14/2021 10:32:00 PM,7300 S MOZART ST,NON-FATAL SHOOTING,SHOOT-JE266314-1,60629,18,CHICAGO LAWN,None,1,...,04B,AGGRAVATED BATTERY (INDEX),041A,AGGRAVATED - HANDGUN,6,2,22,41.760197,-87.694021,POINT (-87.694020941301 41.760196904142)
20032,JE289151,07/03/2021 11:49:00 PM,1000 W 89TH ST,NON-FATAL SHOOTING,SHOOT-JE289151-1,60620,21,AUBURN GRESHAM,Acclivus Washington Heights,2,...,04B,AGGRAVATED BATTERY (INDEX),041A,AGGRAVATED - HANDGUN,7,7,23,41.733063,-87.649051,POINT (-87.649050941301 41.733063095858)
20033,JE290206,07/04/2021 10:44:00 PM,3000 E 79TH PL,NON-FATAL SHOOTING,SHOOT-JE290206-1,60617,7,SOUTH CHICAGO,Claretian Associates South Chicago,2,...,04B,AGGRAVATED BATTERY (INDEX),041A,AGGRAVATED - HANDGUN,7,1,22,41.750657,-87.550559,POINT (-87.550559058699 41.750656904142)
20034,JE292814,07/06/2021 10:36:00 PM,12100 S DOTY AVE,FATAL SHOOTING,HOM-JE292814-1,60628,9,RIVERDALE,None,2,...,00,NON-CRIMINAL,5078,INVESTIGATION,7,3,22,41.673107,-87.600781,POINT (-87.600780941301 41.673106904142)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20036 entries, 0 to 20035
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   CASE_NUMBER                   20036 non-null  object 
 1   DATE                          20036 non-null  object 
 2   BLOCK                         20036 non-null  object 
 3   PRIMARY_TYPE                  20036 non-null  object 
 4   UNIQUE_ID                     20036 non-null  object 
 5   ZIP_CODE                      20036 non-null  int64  
 6   WARD                          20036 non-null  int64  
 7   COMMUNITY_AREA                20036 non-null  object 
 8   STREET OUTREACH ORGANIZATION  20036 non-null  object 
 9   AREA                          20036 non-null  int64  
 10  DISTRICT                      20036 non-null  int64  
 11  BEAT                          20036 non-null  int64  
 12  AGE                           19801 non-null  object 
 13  S

In [5]:
df['STREET OUTREACH ORGANIZATION'].unique()

array(['None', 'PBMR New City', 'TADC Englewood | READI 60621',
       'Project H.O.O.D. Woodlawn', 'READI 60644 | INVC Austin',
       'New Life Centers Brighton Park',
       'READI 60624 | INVC West Garfield Park',
       'UCAN North Lawndale | READI 60623',
       'Claretian Associates South Deering', 'SWOP Chicago Lawn',
       'READI 60644 | INVC West Garfield Park', 'TADC Auburn Gresham',
       'UCAN North Lawndale | Breakthrough East Garfield Park',
       'READI 60623 | New Life Centers South Lawndale | Enlace South Lawndale',
       'READI 60644', 'UCAN Roseland', 'Acclivus Washington Heights',
       'UCAN Riverdale',
       'New Life Centers Lower West Side | Enlace Lower West Side',
       'ALSO Humboldt Park', 'TADC West Englewood | READI 60631',
       'Acclivus Washington Park', 'ALSO Albany Park', 'Acclivus Douglas',
       'INVC Austin', 'Together Chicago Near West Side', 'SWOP Gage Park',
       'ALSO Avondale', 'READI 60624 | ALSO Humboldt Park',
       'Claretian 

In [10]:
df[df['ZIP_CODE']==60626]

,CASE_NUMBER,DATE,BLOCK,PRIMARY_TYPE,UNIQUE_ID,ZIP_CODE,WARD,COMMUNITY_AREA,STREET OUTREACH ORGANIZATION,AREA,...,FBI_CD,FBI_DESCR,IUCR_CD,IUCR_SECONDARY,MONTH,DAY_OF_WEEK,HOUR,LATITUDE,LONGITUDE,LOCATION
28,HZ120293,01/18/2016 11:47:00 PM,6800 N RIDGE BLVD,FATAL SHOOTING,HOM-HZ120293-1,60626,49,ROGERS PARK,None,3,...,01A,HOMICIDE - 1ST OR 2ND DEGREE (INDEX),0110,FIRST DEGREE MURDER,1,2,23,42.005282,-87.679705,POINT (-87.67970465388 42.005281654142)
278,HZ485266,10/23/2016 01:47:00 AM,7300 N SHERIDAN RD,FATAL SHOOTING,HOM-HZ485266-1,60626,49,ROGERS PARK,None,3,...,01A,HOMICIDE - 1ST OR 2ND DEGREE (INDEX),0110,FIRST DEGREE MURDER,10,1,1,42.014677,-87.664889,POINT (-87.664889058699 42.014676904142)
300,JE230465,05/14/2021 10:51:00 PM,7300 N PAULINA ST,NON-FATAL SHOOTING,SHOOT-JE230465-1,60626,49,ROGERS PARK,None,3,...,04B,AGGRAVATED BATTERY (INDEX),041A,AGGRAVATED - HANDGUN,5,6,22,42.014755,-87.672605,POINT (-87.672605 42.0147555)
614,JA238126,04/25/2017 07:00:00 AM,1400 W JONQUIL TER,FATAL SHOOTING,HOM-JA238126-1,60626,49,ROGERS PARK,ONE Northside Rogers Park,3,...,01A,HOMICIDE - 1ST OR 2ND DEGREE (INDEX),0110,FIRST DEGREE MURDER,4,3,7,42.020747,-87.667899,POINT (-87.667899058699 42.020746904142)
637,HZ005025,09/27/2016 04:05:00 AM,1500 W JONQUIL TER,FATAL SHOOTING,HOM-HZ005025-1,60626,49,ROGERS PARK,ONE Northside Rogers Park,3,...,01A,HOMICIDE - 1ST OR 2ND DEGREE (INDEX),0110,FIRST DEGREE MURDER,9,3,4,42.020430,-87.670060,POINT (-87.67006 42.0204305)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19250,JE231417,05/15/2021 08:30:00 PM,1600 W HOWARD ST,NON-FATAL SHOOTING,SHOOT-JE231417-1,60626,49,ROGERS PARK,ONE Northside Rogers Park,3,...,04B,AGGRAVATED BATTERY (INDEX),041A,AGGRAVATED - HANDGUN,5,7,20,42.019292,-87.671705,POINT (-87.67170534612 42.019291654142)
19285,JE234818,05/19/2021 02:17:00 AM,1400 W JONQUIL TER,NON-FATAL SHOOTING,SHOOT-JE234818-1,60626,49,ROGERS PARK,ONE Northside Rogers Park,3,...,03,ROBBERY (INDEX),033B,ATTEMPT ARMED - OTHER FIREARM,5,4,2,42.021208,-87.667695,POINT (-87.66769534612 42.021208345858)
19418,JE248202,05/30/2021 07:17:00 PM,7300 N ROGERS AVE,NON-FATAL SHOOTING,SHOOT-JE248202-1,60626,49,ROGERS PARK,None,3,...,04B,AGGRAVATED BATTERY (INDEX),041A,AGGRAVATED - HANDGUN,5,1,19,42.014588,-87.678545,POINT (-87.67854465388 42.014588345858)
19569,JE264082,06/13/2021 04:12:00 AM,1100 W COLUMBIA AVE,NON-FATAL SHOOTING,SHOOT-JE264082-1,60626,49,ROGERS PARK,None,3,...,04B,AGGRAVATED BATTERY (INDEX),041A,AGGRAVATED - HANDGUN,6,1,4,42.004938,-87.658545,POINT (-87.65854465388 42.004938345858)


In [16]:
df_cases = df.groupby('CASE_NUMBER').count()
df_cases

,DATE,BLOCK,PRIMARY_TYPE,UNIQUE_ID,ZIP_CODE,WARD,COMMUNITY_AREA,STREET OUTREACH ORGANIZATION,AREA,DISTRICT,...,FBI_CD,FBI_DESCR,IUCR_CD,IUCR_SECONDARY,MONTH,DAY_OF_WEEK,HOUR,LATITUDE,LONGITUDE,LOCATION
CASE_NUMBER,,,,,,,,,,,,,,,,,,,,,
HZ005025,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
HZ100035,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
HZ100040,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
HZ100096,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
HZ100171,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JE300480,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
JE300498,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
JE300609,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [26]:
df_cases[df_cases['DATE']>5]

,DATE,BLOCK,PRIMARY_TYPE,UNIQUE_ID,ZIP_CODE,WARD,COMMUNITY_AREA,STREET OUTREACH ORGANIZATION,AREA,DISTRICT,BEAT,AGE,SEX,RACE,FBI_CD,FBI_DESCR,IUCR_CD,IUCR_SECONDARY,MONTH,DAY_OF_WEEK,HOUR,LATITUDE,LONGITUDE,LOCATION
CASE_NUMBER,,,,,,,,,,,,,,,,,,,,,,,,
HZ140230,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8
HZ529307,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
JA129956,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
JA301703,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
JA398576,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
JB322799,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
JB331231,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
JB379745,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8
JB384159,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6


In [27]:
df[df['CASE_NUMBER']=='HZ140230']

,CASE_NUMBER,DATE,BLOCK,PRIMARY_TYPE,UNIQUE_ID,ZIP_CODE,WARD,COMMUNITY_AREA,STREET OUTREACH ORGANIZATION,AREA,DISTRICT,BEAT,AGE,SEX,RACE,FBI_CD,FBI_DESCR,IUCR_CD,IUCR_SECONDARY,MONTH,DAY_OF_WEEK,HOUR,LATITUDE,LONGITUDE,LOCATION
66,HZ140230,02/04/2016 01:00:00 PM,5700 S CALIFORNIA AVE,FATAL SHOOTING,HOM-HZ140230-3,60629,16,GAGE PARK,SWOP Gage Park,1,8,824,30 - 39,F,WWH,01A,HOMICIDE - 1ST OR 2ND DEGREE (INDEX),0110,FIRST DEGREE MURDER,2,5,13,41.79019,-87.693689,POINT (-87.693688547471 41.790189668947)
67,HZ140230,02/04/2016 01:00:00 PM,5700 S CALIFORNIA AVE,NON-SHOOTING HOMICIDE,HOM-HZ140230-4,60629,16,GAGE PARK,SWOP Gage Park,1,8,824,30 - 39,M,WWH,01A,HOMICIDE - 1ST OR 2ND DEGREE (INDEX),0110,FIRST DEGREE MURDER,2,5,13,41.79019,-87.693689,POINT (-87.693688547471 41.790189668947)
68,HZ140230,02/04/2016 01:00:00 PM,5700 S CALIFORNIA AVE,NON-SHOOTING HOMICIDE,HOM-HZ140230-5,60629,16,GAGE PARK,SWOP Gage Park,1,8,824,60 - 69,M,WWH,01A,HOMICIDE - 1ST OR 2ND DEGREE (INDEX),0110,FIRST DEGREE MURDER,2,5,13,41.79019,-87.693689,POINT (-87.693688547471 41.790189668947)
69,HZ140230,02/04/2016 01:00:00 PM,5700 S CALIFORNIA AVE,NON-SHOOTING HOMICIDE,HOM-HZ140230-6,60629,16,GAGE PARK,SWOP Gage Park,1,8,824,50 - 59,F,WWH,01A,HOMICIDE - 1ST OR 2ND DEGREE (INDEX),0110,FIRST DEGREE MURDER,2,5,13,41.79019,-87.693689,POINT (-87.693688547471 41.790189668947)
687,HZ140230,02/04/2016 01:00:00 PM,5700 S CALIFORNIA AVE,NON-SHOOTING HOMICIDE,HOM-HZ140230-1,60629,16,GAGE PARK,SWOP Gage Park,1,8,824,0 - 19,M,WWH,01A,HOMICIDE - 1ST OR 2ND DEGREE (INDEX),0110,FIRST DEGREE MURDER,2,5,13,41.79019,-87.693689,POINT (-87.693688547471 41.790189668947)
3941,HZ140230,02/02/2016 12:00:00 PM,5700 S CALIFORNIA AVE,NON-FATAL SHOOTING,SHOOT-HZ140230-2,60629,16,GAGE PARK,SWOP Gage Park,1,8,824,60 - 69,M,WWH,04B,AGGRAVATED BATTERY (INDEX),041A,AGGRAVATED: HANDGUN,2,3,12,41.79019,-87.693689,POINT (-87.693688547471 41.790189668947)
3942,HZ140230,02/02/2016 12:00:00 PM,5700 S CALIFORNIA AVE,NON-FATAL SHOOTING,SHOOT-HZ140230-5,60629,16,GAGE PARK,SWOP Gage Park,1,8,824,30 - 39,M,WWH,04B,AGGRAVATED BATTERY (INDEX),041A,AGGRAVATED: HANDGUN,2,3,12,41.79019,-87.693689,POINT (-87.693688547471 41.790189668947)
17074,HZ140230,02/04/2016 01:00:00 PM,5700 S CALIFORNIA AVE,NON-SHOOTING HOMICIDE,HOM-HZ140230-2,60629,16,GAGE PARK,SWOP Gage Park,1,8,824,0 - 19,M,WWH,01A,HOMICIDE - 1ST OR 2ND DEGREE (INDEX),0110,FIRST DEGREE MURDER,2,5,13,41.79019,-87.693689,POINT (-87.693688547471 41.790189668947)
